In [76]:
import mne
import os
from mne.time_frequency import psd_array_multitaper
import numpy as np
import pandas as pd
from scipy.stats import entropy, skew, kurtosis
import csv



In [77]:
# field names 
fields = ['File Name', 'Start', 'End'] 

# data rows of csv file 
rows = [['chb08_02.edf',	2670 ,	2841],
        ['chb08_05.edf',	2856 ,	3046 ],
        ['chb08_11.edf',	2988 ,	3122 ],
        ['chb08_13.edf',	2417 ,	2577 ],
        ['chb08_21.edf',	2083 ,	2347 ],
]


# name of csv file 
filename = "chb08_seizure.csv"

# writing to csv file 
with open(filename, 'w') as csvfile: 
        # creating a csv writer object 
        csvwriter = csv.writer(csvfile) 

        # writing the fields 
        csvwriter.writerow(fields) 

        # writing the data rows 
        csvwriter.writerows(rows)

In [78]:
freq_bands = {'delta': [0.5, 4], 'theta': [4, 7], 'alpha': [7, 15], 'beta': [15, 30]}

In [79]:
def extract_spectral_features(raw, freq_bands):
    spectral_features = []
    raw.filter(1, 120)
    raw.set_eeg_reference('average')
    eeg_picks = mne.pick_types(raw.info, meg=False, eeg=True)
    psds, freqs = psd_array_multitaper(raw.get_data()[eeg_picks], sfreq=raw.info['sfreq'], fmin=0.5, fmax=30) # type: ignore

    for band, freq_range in freq_bands.items():
        band_data = raw.copy().filter(freq_range[0], freq_range[1], picks=eeg_picks).get_data()

        if np.isnan(band_data).any():
            print(f"No data available for {band} band.")
            continue

        band_power = np.mean(np.abs(band_data) ** 2, axis=1)
        band_entropy = entropy(np.abs(band_data) ** 2, axis=1)
        band_skewness = skew(band_data, axis=1)

        # spectral_features += ["freq",band_power, band_entropy, band_skewness]
        # spectral_features.append(band_power[0][ch],)
        for ch in range(23):
            # print(band_power.shape)
            spectral_features.append(band_power[ch])
            spectral_features.append(band_entropy[ch])
            spectral_features.append(band_skewness[ch])
            
    return spectral_features

def extract_time_features(raw):
    time_features = []

    eeg_picks = mne.pick_types(raw.info, meg=False, eeg=True)
    eeg_data = raw.get_data()[eeg_picks]

    for channel_data in eeg_data:
        time_features.append(np.mean(channel_data)) 

        time_features.append(np.std(channel_data))

        time_features.append(kurtosis(channel_data))
            
        time_features.append(np.ptp(channel_data,axis=-1))

        time_features.append(np.var(channel_data,axis=-1))

        time_features.append(np.min(channel_data,axis=-1))

        time_features.append(np.max(channel_data,axis=-1))

        time_features.append(np.argmin(channel_data,axis=-1))

        time_features.append(np.argmax(channel_data,axis=-1))

        time_features.append(np.sqrt(np.mean(channel_data ** 2,axis=-1)))  #root mean sqaured #type: ignore

    return time_features

In [80]:
def extract_features(patient, file, start=0, end=3599, window=20, seizure_start=[0], seizure_end=[0]):

    
    raw = mne.io.read_raw_edf(f'{patient}/{file}', preload=True)
    temp=start


    while temp<end:
        all_features = [file,temp,temp+window]
        # create a copy of the original object
        raw_copy = raw.copy()

        # apply slicing on the copy
        if (temp+window)<end:
            current_data = raw_copy.crop(tmin=temp, tmax=(temp+window))
        else:
            current_data = raw_copy.crop(tmin=temp, tmax=end)


        spectral_features = extract_spectral_features(current_data, freq_bands)
        time_features = extract_time_features(current_data)
        
        all_features.extend( spectral_features + time_features)

        target=0
        for seiz_start,seiz_end in zip(seizure_start,seizure_end):
            if temp in range(seiz_start,(seiz_end-1)):
                target = 1

        # if temp in range(seizure_start,(seizure_end-1)):
        #     target = 1
        # else:
        #     target = 0
        all_features.append(target) 
        print('***********************************************')
        print(len(all_features))
        print(f"start={temp}, end={temp+window}")
        print(all_features)

        temp+=window
        
        data = [float(item) if isinstance(item, np.float64) else item for item in all_features] # type: ignore
        print('***********************************************')
        print(len(data))
        # print(len(data[0]))

        with open(f'{patient}_features.csv', 'a', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(data)
        

In [81]:
list_channels=['ch1','ch2','ch3','ch4','ch5','ch6','ch7','ch8','ch9','ch10','ch11',
               'ch12','ch13','ch14','ch15','ch16','ch17','ch18','ch19','ch20','ch21','ch22','ch23']
features=['delta_power','delta_entropy','delta_skewness',
            'theta_power','theta_entropy','theta_skewness',
            'alpha_power','alpha_entropy','alpha_skewness',
            'beta_power','beta_entropy','beta_skewness',
            'mean','std','kurtosis','ptp','var','max','min','argmax','argmin','rms']

new_header=['id','start','end']
for item in list_channels:
    for item2 in  features:
        new_header.append(item+'_'+item2)
new_header.append('target')
print(len(new_header))
with open(f'CH03_features.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(new_header)

510


In [82]:
# set path to the seizure.csv file
seizure_csv = 'chb03_seizure.csv'


# read the seizure.csv file as a pandas dataframe
seizure_df = pd.read_csv(seizure_csv)

# set path to the chb03 folder
chb_folder = 'CH03'

# get a list of all the files in the chb03 folder with the '.edf' extension
edf_files_list = [f for f in os.listdir(chb_folder) if f.endswith('.edf')]
# channel_names=['FP1-F7', 'F7-T7', 'T7-P7', 'P7-O1', 'FP1-F3', 'F3-C3', 'C3-P3',
#                 'P3-O1', 'FP2-F4', 'F4-C4', 'C4-P4', 'P4-O2', 'FP2-F8', 'F8-T8',
#                 'T8-P8', 'P8-O2', 'FZ-CZ', 'CZ-PZ', 'P7-T7', 'T7-FT9', 'FT9-FT10',
#                 'FT10-T8','T8-P8']
# list_channels=['ch1','ch2','ch3','ch4','ch5','ch6','ch7','ch8','ch9','ch10','ch11',
#                'ch12','ch13','ch14','ch15','ch16','ch17','ch18','ch19','ch20','ch21','ch22','ch23']
# features=['delta_power','delta_entropy','delta_skewness',
#             'theta_power','theta_entropy','theta_skewness',
#             'alpha_power','alpha_entropy','alpha_skewness',
#             'beta_power','beta_entropy','beta_skewness',
#             'mean','std','kurtosis','ptp','var','max','min','argmax','argmin','rms']

# new_header=[]
# for item in list_channels:
#     for item2 in  features:
#         new_header.append(item+item2)

# loop through each file in the chb03 folder
for file in edf_files_list:
    # check if the file exists in the seizure.csv file
    if file in seizure_df['File Name'].values:
        # if the file exists, get the corresponding start and end times from the CSV file
        row = seizure_df.loc[seizure_df['File Name'] == file]
        seizure_start = row['Start'].values
        seizure_end = row['End'].values
        extract_features(patient=chb_folder ,file=file,start=0,end=3599,window=20,seizure_start=seizure_start,seizure_end=seizure_end)
    else:
        # if the file does not exist, assign both variables a value of zero
        seizure_start = 0
        seizure_end = 0
        extract_features(patient=chb_folder ,file=file,start=0,end=3599,window=20,seizure_start=seizure_start,seizure_end=seizure_end)
    
    # do something with the seizure_start and seizure_end variables
    # for example, print them to the console
    print(f'File: {file}, Seizure Start: {seizure_start}, Seizure End: {seizure_end}')

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'CH03'

In [ ]:

def read_eeg_files(directory):
    eeg_data = []

    for filename in os.listdir(directory):
        if filename.endswith('.edf'):
            filepath = os.path.join(directory, filename)
            raw = mne.io.read_raw_edf(filepath, preload=True)
            eeg_data.append(raw)

    return eeg_data

def extract_spectral_features(raw, freq_bands):
    print("********************************************")
    print(type(raw))
    spectral_features = []

    raw.filter(1, 120)
    raw.set_eeg_reference('average')
    eeg_picks = mne.pick_types(raw.info, meg=False, eeg=True)
    psds, freqs = psd_array_multitaper(raw.get_data()[eeg_picks], sfreq=raw.info['sfreq'], fmin=0.5, fmax=30) # type: ignore

    for band, freq_range in freq_bands.items():
        band_data = raw.copy().filter(freq_range[0], freq_range[1], picks=eeg_picks).get_data()

        if np.isnan(band_data).any():
            print(f"No data available for {band} band.")
            continue

        band_power = np.mean(np.abs(band_data) ** 2, axis=1)
        band_entropy = entropy(np.abs(band_data) ** 2, axis=1)
        band_skewness = skew(band_data, axis=1)

        spectral_features += [band_power, band_entropy, band_skewness]


    return spectral_features

def extract_time_features(raw):
    time_features = []

    eeg_picks = mne.pick_types(raw.info, meg=False, eeg=True)
    eeg_data = raw.get_data()[eeg_picks]

    for channel_data in eeg_data:
        channel_mean = np.mean(channel_data)

        channel_std = np.std(channel_data)

        channel_kurtosis = kurtosis(channel_data)
            
        channel_ptp = np.ptp(channel_data,axis=-1)

        channel_var = np.var(channel_data,axis=-1)

        channel_min = np.min(channel_data,axis=-1)

        channel_max = np.max(channel_data,axis=-1)

        channel_argmin = np.argmin(channel_data,axis=-1)

        channel_argmax = np.argmax(channel_data,axis=-1)

        channel_rms= np.sqrt(np.mean(channel_data ** 2,axis=-1))  #root mean sqaured #type: ignore


        time_features +=[channel_mean,channel_std,channel_kurtosis,channel_ptp,channel_var,channel_min,channel_max,channel_argmin,channel_argmax,channel_rms]

    return time_features


def save_to_csv(data, filename):
    # Convert numpy.float64 objects to regular floats
    data = [[float(item) if isinstance(item, np.float64) else item for item in row] for row in data] # type: ignore

    with open(filename, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerows(data)

In [ ]:

# Example usage:
directory = 'CH03'
freq_bands = {'delta': [0.5, 4], 'theta': [4, 7], 'alpha': [7, 15], 'beta': [15, 30]}

eeg_data = read_eeg_files(directory)

all_features = []

for raw in eeg_data:
    print("********************************************")
    print(type(raw))
    spectral_features = extract_spectral_features(raw, freq_bands)
    time_features = extract_time_features(raw)

    # Combine spectral and time features into a single list
    all_features.append(spectral_features + time_features)

# Save features to a CSV file
save_to_csv(all_features, 'features.csv')


In [ ]:

import pandas as pd
technologies = {
    'Courses':["PySpark","PySpark","Hadoop","Python","pandas"],
    'Fee' :[20000,25000,26000,22000,24000],
    'Duration':['30day','40days','35days','40days','60days'],
    'Discount':[1000,2300,1200,2500,2000]
              }
# index_labels=['r1','r2','r3','r4','r5']
df = pd.DataFrame(technologies)
print(df)
print(df.loc[['PySpark']])
# Outputs
#r1    Spark  20000    30day      1000
#r2  PySpark  25000   40days      2300
#r3   Hadoop  26000   35days      1200
#r4   Python  22000   40days      2500
#r5   pandas  24000   60days      2000


In [ ]:
# field names 
fields = ['File Name', 'Start', 'End'] 

# data rows of csv file 
rows = [['chb08_02.edf',	2670 ,	2841],
        ['chb08_02.edf',	2856 ,	3046 ],
        ['chb08_11.edf',	2988 ,	3122 ],
        ['chb08_13.edf',	2417 ,	2577 ],
        ['chb08_21.edf',	2083 ,	2347 ],
]


# name of csv file 
filename = "trail.csv"

# writing to csv file 
with open(filename, 'w') as csvfile: 
        # creating a csv writer object 
        csvwriter = csv.writer(csvfile) 

        # writing the fields 
        csvwriter.writerow(fields) 

        # writing the data rows 
        csvwriter.writerows(rows)

In [ ]:

seizure_df = pd.read_csv('trail.csv')
# print(seizure_df)

    # if the file exists, get the corresponding start and end times from the CSV file
row = seizure_df.loc[seizure_df['File Name'] == 'chb08_02.edf']
seizure_start = row['Start'].values[0]
seizure_end = row['End'].values[0]
print(row['Start'].values)
# print(f'File: {file}, Seizure Start: {seizure_start}, Seizure End: {seizure_end}')

[2670 2856]
